Final Project 
======

Kaggle Competition 
-----

For this project I chose to do the active competition [San Francisco Crime Classification](https://www.kaggle.com/c/sf-crime/). I'll test out a number of different algorithms with test data, but this seems like probably RandomForest a Neural Network 2 levels deep will have the best results. I am interested in this type of analysis as it is data science that contributes to the common good.

#### Part 1 - Submit first (bad) try

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import LabelEncoder

In [6]:
train_raw = pd.read_csv('Data/train.csv')
test_raw = pd.read_csv('Data/test.csv')
sample_submission = pd.read_csv('Data/sampleSubmission.csv')
print train_raw.shape

(878049, 9)


In [8]:
train_raw.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [9]:
test_raw.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [10]:
sample_submission.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [82]:
train_data\
,train_labels = np.array(train_raw[['DayOfWeek'\
                                     ,'PdDistrict'\
                                     ,'Address'\
                                     ,'X'\
                                     ,'Y']][2001:])\
                            ,np.array(train_raw['Category'][2001:])
dev_data\
,dev_labels = np.array(train_raw[['DayOfWeek'\
                                 ,'PdDistrict'\
                                 ,'Address'\
                                 ,'X'\
                                 ,'Y']][:1000])\
                            ,np.array(train_raw['Category'][:1000])
dev_test_data\
,dev_test_labels = np.array(train_raw[['DayOfWeek'\
                                 ,'PdDistrict'\
                                 ,'Address'\
                                 ,'X'\
                                 ,'Y']][1001:2000])\
                            ,np.array(train_raw['Category'][1001:2000])
test_data = np.array(test_raw[['DayOfWeek'\
                     ,'PdDistrict'\
                     ,'Address'\
                     ,'X'\
                     ,'Y']])

In [64]:
rf = RandomForestClassifier() #leaving with default settings for now

Data needs to be preprocessed for RF in sklearn

In [73]:
def recode(df):
    for i in range(df.shape[1]):
        le = LabelEncoder()
        le.fit(df[:,i])
        df[:,i] = le.transform(df[:,i])
    return df

def recode_labels(df):
    le = LabelEncoder()
    le.fit(df)
    return le

dev_data = recode(dev_data)
dev_label_le = recode_labels(np.concatenate((dev_labels,dev_test_labels),axis=1))
dev_labels = dev_label_le.transform(dev_labels)
dev_test_data = recode(dev_test_data)
dev_test_labels = dev_label_le.transform(dev_test_labels)

In [74]:
print dev_data[:5,],'\n',dev_labels[:5,]

[[2 4 648 268 370]
 [2 4 648 268 370]
 [2 4 691 275 670]
 [2 4 168 259 673]
 [2 5 67 184 337]] 
[27 16 16 12 12]


Run model

In [76]:
rf.fit(dev_data,dev_labels)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [77]:
predictions = rf.predict(dev_test_data)

In [78]:
actual_predictions = dev_label_le.inverse_transform(predictions)
actual_labels = dev_label_le.inverse_transform(dev_test_labels)
print actual_predictions[:5],'\n',actual_labels[:5]

['VEHICLE THEFT' 'LARCENY/THEFT' 'SUSPICIOUS OCC' 'OTHER OFFENSES'
 'OTHER OFFENSES'] 
['OTHER OFFENSES' 'LARCENY/THEFT' 'LARCENY/THEFT' 'WARRANTS' 'WARRANTS']


In [79]:
accuracy = np.sum((actual_predictions == actual_labels)*1.0)/len(dev_labels)
print 'Dev Accuracy: {0}%'.format(accuracy*100)

Dev Accuracy (overfit): 21.9%


Train with real data

In [83]:
train_data, test_data = recode(train_data), recode(test_data)
train_le = recode_labels(train_labels)
train_labels = train_le.transform(train_labels)

In [84]:
rf = RandomForestClassifier()
rf.fit(train_data, train_labels)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

Make predictions and reformat for submission

In [85]:
predictions = rf.predict(test_data)
actual_predictions = train_le.inverse_transform(predictions)

In [86]:
l = len(test_data)
submission = np.zeros((l,40),dtype=np.int32)
submission[:,0] = range(l)
cols = train_le.classes_ 
for i,c in enumerate(cols):
    submission[:,i+1] = actual_predictions == c

In [87]:
submission_cols = ['Id']
submission_cols.extend(cols)
submission_df = pd.DataFrame(submission,columns=submission_cols)
submission_df.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Save data to csv file for submission

In [88]:
submission_df.to_csv('Data/submission_file1.csv',index=False)

#### Part 2 - Tests to increase accuracy

First attempt: Rank 208, score 26.95890, tested accuracy of ~22%.

##### Add time of day 

In [89]:
# Reload data 
train_raw = pd.read_csv('Data/train.csv')
test_raw = pd.read_csv('Data/test.csv')

In [90]:
def segment_time1(ts):
    ''' Segment time of day '''
    if ts.hour<=4: return 1
    elif ts.hour<=8: return 2
    elif ts.hour<=12: return 3
    elif ts.hour<=16: return 4
    elif ts.hour<=20: return 5
    elif ts.hour<=24: return 6
def segment_time2(ts):
    ''' Return only hour '''
    return ts.hour
def dayofmonth(ts):
    ''' Return day of month '''
    return ts.day
def yearofcrime(ts):
    ''' Return the year '''
    return ts.year

segment_time1 = np.vectorize(segment_time1,otypes=[np.int64])
segment_time2 = np.vectorize(segment_time2,otypes=[np.int64])
dayofmonth = np.vectorize(dayofmonth,otypes=[np.int64])
yearofcrime = np.vectorize(yearofcrime,otypes=[np.int64])

train_raw['Dates'] = pd.to_datetime(train_raw['Dates'])
test_raw['Dates'] = pd.to_datetime(test_raw['Dates'])

train_raw['DaySegment'] = train_raw['Dates'].apply(segment_time1)
test_raw['DaySegment'] = test_raw['Dates'].apply(segment_time1)

train_raw['TimeOfDay'] = train_raw['Dates'].apply(segment_time2)
test_raw['TimeOfDay'] = test_raw['Dates'].apply(segment_time2)

train_raw['DayOfMonth'] = train_raw['Dates'].apply(dayofmonth)
test_raw['DayOfMonth'] = test_raw['Dates'].apply(dayofmonth)

train_raw['Year'] = train_raw['Dates'].apply(yearofcrime)
test_raw['Year'] = test_raw['Dates'].apply(yearofcrime)

In [91]:
g = train_raw.groupby('PdDistrict')
g['PdDistrict'].agg(len)

PdDistrict
BAYVIEW        89431
CENTRAL        85460
INGLESIDE      78845
MISSION       119908
NORTHERN      105296
PARK           49313
RICHMOND       45209
SOUTHERN      157182
TARAVAL        65596
TENDERLOIN     81809
Name: PdDistrict, dtype: int64

Creating a function here that can be reused across other tests when adding / removing features

In [94]:
# Run test on dev data
def analyze_impact(raw_data, old_fields, new_fields,\
                   label_field, s=10000, n=100):
    ''' Iterate over many RF tries to 
        see how change impacts results 
    '''
    raw_data = raw_data.iloc[\
                    np.random.permutation(len(raw_data))]
    dev_data_old\
    ,dev_data_new\
    ,dev_labels = np.array(raw_data[old_fields][:s])\
                    ,np.array(raw_data[new_fields][:s])\
                    ,np.array(raw_data[label_field][:s])
    test_data_old\
    ,test_data_new\
    ,test_labels = np.array(raw_data[old_fields][s+1:s+5000])\
                    ,np.array(raw_data[new_fields][s+1:s+5000])\
                    ,np.array(raw_data[label_field][s+1:s+5000])
    
    old_accuracies = []
    new_accuracies = []
    def measure(d, t, n, out):
        for i in range(n):
            # Initialize empty model 
            rf = RandomForestClassifier() 

            # Recode 
            d = recode(d)
            t = recode(t)
            d_le = recode_labels(np.concatenate((\
                                                dev_labels\
                                                ,test_labels)\
                                                   ,axis=1))
            d_l = d_le.transform(dev_labels)
            t_l = d_le.transform(test_labels)

            # Fit model 
            rf.fit(d, d_l)

            # Predict 
            predictions = rf.predict(t)

            # Measure accuracy
            accuracy = np.sum((predictions == \
                               t_l)*1.0)/len(t_l)

            # add result 
            out.append(accuracy)
    
    measure(dev_data_old.copy(), test_data_old.copy(), n, old_accuracies)
    measure(dev_data_new.copy(), test_data_new.copy(), n, new_accuracies)
    m_old = np.mean(old_accuracies)
    s_old = np.std(old_accuracies)*1.0/np.sqrt(n)
    m_new = np.mean(new_accuracies)
    s_new = np.std(new_accuracies)*1.0/np.sqrt(n)
    print 'Old Accuracy: {0}% +/- {1}%\nNew Accuracy: {2}% +/- {3}%'\
        .format(round(m_old*100,2)
                ,round(1.96*s_old*100,2)
                ,round(m_new*100,2)
                ,round(1.96*s_new*100,2)
        )

In [95]:
s = 100000 
old_fields = ['PdDistrict'\
              ]
new_fields = ['PdDistrict'\
              ,'DaySegment'\
             ]
nt = train_raw.copy()
analyze_impact(nt, old_fields, new_fields, \
               'Category', s, 30)

Old Accuracy: 22.4% +/- 0.0%
New Accuracy: 22.56% +/- 0.02%


Test Notes:
* Adding time of day increased accuracy by ~2 percentage points, until I implemented random shuffling prior to testing - then the impact of adding time of day was negligable or only slightly positive in most trials. 
* Time of Day as an hour originally performed better than 4-hour blocks; however, after implementing random shuffling the day segment field performed better across multiple tries.
* Adding day of month reduced model performance due to overfitting. 
* Adding year reduced model performance due to overfitting. 
* Adding Address significantly reduces model accuracy due to overfitting
* Day of week significantly reduces model accuracy due to overfitting
* Lat/Lon significantly reduces model accuracy due to overfitting
* The actual expected accuracy is still very bad (<30%)

Other notes:
* This is a funny dataset because the only raw feature that works for prediction is Police District 
* It's a good example of how a model can easily be overfit with useless features
* The features provided are probably not enough to accurately predict category of crime; however, lat/lon is probably quite useful with some feature engineering so that it will generalize better

##### Add area of crime in place of lat/lon

Looking to test out a couple options for this feature. Currently, by using the exact location of the crime, the model does not generalize very well. A couple options are available: 1) utilize publicly available datasets on city neighborhoods to classify each event into a more generalized city segment, perhaps based on income levels, 2) use a clustering algorithm like K-means to segment the training data into clusters where there are crime "hotspots" and add one of these as a feature (replacing lat/lon and potentially address - will test the removal of features separately). 

I will first attempt #2 while looking for a good option for #1.

##### Get data from other sources

SF OpenData has a ton of supplemental data sources that will be great to try out for this effort.  

In [7]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap, cm
import cartopy.io.shapereader as shpreader
import fiona
import numpy as np
import matplotlib.pyplot as plt
import pysal 
from pyproj import Proj
from pysal.cg.shapes import Point
from pysal.cg.locators import PolygonLocator, PointLocator

In [8]:
# http://spatialreference.org/ref/epsg/2227/
p = Proj('+proj=lcc +init=EPSG:2227 +datum=NAD83 +units=us-ft +no_defs',preserve_units=True)
convert_vals = np.vectorize(lambda x,y: p(x,y))
convert_vals_inv = np.vectorize(lambda x,y: p(x,y,inverse=True))

In [9]:
shpfilename = 'Data/SanFranciscoElectricityUse/SanFranciscoElectricityUse.shp'
shxfilename = 'Data/SanFranciscoElectricityUse/SanFranciscoElectricityUse.shx'

In [112]:
# test reading with pysal (for analysis)
shp = pysal.open(shpfilename,'r')
# arcgis_dbf = pysal.open('Data/SanFranciscoElectricityUse/SanFranciscoElectricityUse.dbf','r','arcgis_dbf')
print type(shp[0])
shp.close()

<class 'pysal.cg.shapes.Polygon'>


In [97]:
# test reading with fiona
shp = fiona.open(shpfilename)
print shp.meta,'\n'
print next(shp)['properties']
shp.close()

{'crs': {u'lon_0': -120.5, u'datum': u'NAD83', u'y_0': 500000.0000000001, u'no_defs': True, u'proj': u'lcc', u'x_0': 2000000, u'units': u'us-ft', u'lat_2': 38.43333333333333, u'lat_1': 37.06666666666667, u'lat_0': 36.5}, 'driver': u'ESRI Shapefile', 'schema': {'geometry': 'Polygon', 'properties': OrderedDict([(u'kWh_pC', 'int:9'), (u'kWh', 'int:9'), (u'Zip', 'int:9'), (u'Pop2010_zc', 'int:9')])}} 

OrderedDict([(u'kWh_pC', 1435), (u'kWh', 44750852), (u'Zip', 94102), (u'Pop2010_zc', 31176)])


In [98]:
# test reading with catropy
shp = shpreader.Reader(shpfilename)
countries = shp.records()
country = next(countries)
print country.attributes,\
    convert_vals_inv(country.bounds[0],country.bounds[1]),\
    convert_vals_inv(country.bounds[2],country.bounds[3])

{'Pop2010_zc': 31176, 'kWh': 44750852, 'kWh_pC': 1435, 'Zip': 94102} (array(-122.42974011242651), array(37.769516792469275)) (array(-122.40597462798799), array(37.789287353962244))


Convert X,Y values into same scale as shapefile

In [137]:
# Test converting X,Y into lat/lon coords
t = train_raw[:5]
t['New_X'], t['New_Y'] = \
    convert_vals(t['X'],t['Y'])
print t

                 Dates        Category                      Descript  \
0  2015-05-13 23:53:00        WARRANTS                WARRANT ARREST   
1  2015-05-13 23:53:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
2  2015-05-13 23:33:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
3  2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   
4  2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   

   DayOfWeek PdDistrict      Resolution                    Address  \
0  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
1  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
2  Wednesday   NORTHERN  ARREST, BOOKED  VANNESS AV / GREENWICH ST   
3  Wednesday   NORTHERN            NONE   1500 Block of LOMBARD ST   
4  Wednesday       PARK            NONE  100 Block of BRODERICK ST   

            X          Y           New_X           New_Y  
0 -122.425892  37.774599  6005079.000336  2110238.000352  
1 -122.425892  37.774599  60

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [10]:
train_raw['New_X'], train_raw['New_Y'] = \
    convert_vals(train_raw['X'],train_raw['Y'])

In [11]:
train_raw[['X','New_X','Y','New_Y']].head()

,X,New_X,Y,New_Y
0,-122.425892,6005079.000336,37.774599,2110238.000352
1,-122.425892,6005079.000336,37.774599,2110238.000352
2,-122.424363,6005714.000336,37.800414,2119627.000352
3,-122.426995,6004957.064992,37.800873,2119809.494638
4,-122.438738,6001343.956404,37.771541,2109201.621496


Create array with shapefile properties

In [12]:
shp = fiona.open(shpfilename)
l = np.zeros(len(shp))
Pop2010_zc,kWh,kWh_pC,Zip = \
    np.zeros(len(shp)),np.zeros(len(shp)),\
    np.zeros(len(shp)),np.zeros(len(shp))
for i,s in enumerate(shp):
    Pop2010_zc[i] = s['properties']['Pop2010_zc']
    kWh[i] = s['properties']['kWh']
    kWh_pC[i] = s['properties']['kWh_pC']
    Zip[i] = s['properties']['Zip']
shp.close()

Find appropriate electricity values to assign to each observation in training data

In [40]:
def make_points(x):
    ''' Create list of Points objects
    '''
    out = []
    n = x.shape[0]
    for i in range(n):
        out.append(Point((x['New_X'][i],x['New_Y'][i])))
    return out
points = make_points(train_raw[['New_X','New_Y']])

In [ ]:
shp = pysal.open(shpfilename,'r')
pl = PolygonLocator([p for p in shp])
shp.close()
def return_poly_id(x):
    ''' Find the polygon that 
        contains each point
    '''
    try:
        return pl.contains_point(x)[0].id
    except IndexError:
        return -1
poly_ids = np.zeros(len(points),dtype=np.int8)
for i,p in enumerate(points):
    if i%100000==0: print 'running {0} row'.format(i)
    poly_ids[i] = return_poly_id(p)

# Save data because this step takes >1hr to finish
poly_ids_df = pd.DataFrame({'elect_poly_id':poly_ids})
poly_ids_df.to_csv('Data/poly_ids.csv',index=False)

#### Part 3 - Error Diagnosis